# LAB 3: How to setup a project from Scratch

# Step 1: Clone your project from Github

In [1]:
# !git clone https://github.com/lambdavi/mldl_project_skeleton.git
!git clone https://github.com/MatteoPaglia/speech-emotion-recognition-25.git

Cloning into 'speech-emotion-recognition-25'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 17 (delta 4), reused 0 (delta 0), pack-reused 11 (from 2)
Receiving objects: 100% (17/17), 5.47 KiB | 2.74 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [2]:
!ls

sample_data  speech-emotion-recognition-25


In [3]:
# %cd mldl_project_skeleton
%cd speech-emotion-recognition-25

/content/speech-emotion-recognition-25


In [4]:
!ls

checkpoints  dataset  models	 requirements.txt  utils
data	     eval.py  README.md  train.py


# Step 2: Packages Installation


In [ ]:
!pip install -r requirements.txt

# Step 3: Dataset Setup
## Different options
- First one is downloading using a script that places the data in the download folder (usually recommended)
- Second one is uploading the dataset to your personal/institutional Google Drive and load it from there ([Read More](https://saturncloud.io/blog/google-colab-how-to-read-data-from-my-google-drive/))
- Place the download script directly here on colab

You are free to do as you please in this phase.


In [ ]:
# !python utils/download_dataset.py

# Step 4: Train your model and visualize training

In [6]:
%env WANDB_API_KEY="7ade30086de7899bed412e3eb5c2da065c146f90"

env: WANDB_API_KEY="7ade30086de7899bed412e3eb5c2da065c146f90"


In [ ]:
!python train.py

# Step 5: Evaluate your model

1.   List item
2.   List item



In [ ]:
!python eval.py